# 대전시 대중교통 통행수요 예측 - 데이터 분석
## 1. 데이터 로드 및 기본 전처리

In [ ]:
import os
import time
import pandas as pd
import sklearn
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rc('font', family='AppleGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

 ## 2. 데이터 전처리
    ### 2.1 불필요한 컬럼 제거

In [ ]:
# 2. 데이터 전처리
## 2.1 데이터 로드 및 불필요한 컬럼 제거
data = pd.read_csv('/Users/PDG/Desktop/머신러닝 프로젝트/train_data_add_gid_weather (1).csv')

columns_to_drop = ['DATETIME', '해면기압 QC플래그', '현지기압 QC플래그',
                   '적설(cm)', '3시간신적설(cm)',
                   '최저운고(100m )', '일사(MJ/m2)',
                   '기온 QC플래그', '강수량 QC플래그',
                   '풍속 QC플래그', '풍향 QC플래그',
                   '습도 QC플래그', '일조(hr)',
                   '일조 QC플래그', '일사 QC플래그',
                   '운형(운형약어)', '지면상태(지면상태코드)',
                   '현상번호(국내식)', '지면온도 QC플래그', 
                   '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', 
                   '해면기압(hPa)', '전운량(10분위)', '지점명', '지점', 'week', 'holiday',
                   '풍속(m/s)', '풍향(16방위)', '습도(%)', '중하층운량(10분위)', '시정(10m)', 
                   '20cm 지중온도(°C)', '30cm 지중온도(°C)']

data = data.drop(columns_to_drop, axis=1)

## 2.2 데이터 분할 (Train/Test Set)
# 시계열 특성을 고려한 테스트셋 분리
undesired_dates = ['2023-08-24', '2023-08-23', '2023-08-22']
train_set = data[~data['DATE'].isin(undesired_dates)]

# 테스트셋 (8월 마지막주 3일치 데이터)
desired_dates = ['2023-08-24', '2023-08-23', '2023-08-22']
test_set = data[data['DATE'].isin(desired_dates)]

## 2.3 상관관계 분석
# 상관관계 매트릭스 생성
corr_matrix = data.corr()
corr_matrix["RIDE_DEMAND"].sort_values(ascending=False)

# 시각화 설정
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=10)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=7)
plt.rc('ytick', labelsize=7)

# 히스토그램 
data.hist(bins=50, figsize=(20, 13))
plt.show()

# 누락된 값 확인
missing_values = data.isnull().sum()
missing_columns = missing_values[missing_values > 0]
print("누락된 값이 있는 칼럼:")
print(missing_columns)

# 3. 피처 엔지니어링 및 데이터 변환
## 3.1 데이터 준비

In [ ]:
# 트레인셋과 타겟 변수 분리
train = train_set.drop(RIDE_DEMAND, axis=1)   
data_labels = train_set[RIDE_DEMAND].copy()  

# 테스트셋과 타겟 변수 분리
test = train_set.drop(RIDE_DEMAND, axis=1)   
test_labels = test_set[RIDE_DEMAND].copy()

## 3.2 데이터 변환 파이프라인 구축
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# 범주형 특성 변환
categorical_features = ['gid', 'DATE', 'TIME']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

# 수치형 특성 변환
numeric_features = ['ALIGHT_DEMAND', '기온(°C)', '강수량(mm)', '지면온도(°C)', 
                   '5cm 지중온도(°C)', '10cm 지중온도(°C)', 'total_population', 
                   'total_building', 'old_population', 'working_population', 
                   'building_to_land_ratio', 'floor_area_ratio', 'bus_count']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('scaler', StandardScaler())
])

# 전체 변환 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 최종 파이프라인
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# 데이터 변환 적용
df_prepared = pipeline.fit_transform(train)
test_df_prepared = pipeline.fit_transform(test)

In [ ]:
# 4. AutoGluon을 사용한 모델링
from autogluon.tabular import TabularPredictor

## 4.1 AutoGluon 학습 설정
predictor = TabularPredictor(
    label='RIDE_DEMAND',  # 타겟 변수
    eval_metric='rmse'    # 평가 지표
).fit(
    train_data=train_set,
    time_limit=3600,      # 학습 시간 제한 (1시간)
    presets='best_quality'  # 최고 품질 설정
)

## 4.2 모델 성능 평가
# 리더보드 확인
leaderboard = predictor.leaderboard(test_set, silent=True)
print("Model Leaderboard:")
print(leaderboard)

# 테스트 세트에 대한 예측
predictions = predictor.predict(test_set)

## 4.3 성능 지표 계산
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(test_labels, predictions))
r2 = r2_score(test_labels, predictions)

print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

## 4.4 특성 중요도 확인
feature_importance = predictor.feature_importance(test_set)
print("\nFeature Importance:")
print(feature_importance)